In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [ ]:
# from transformers import AutoModelForCausalLM
# from utils.torch_utils import count_parameters
# import torch

# model_path="/home/golympie/ai-toolbox/pruned_models/deepseek_coder_v2_lite_instruct_fused_16/"

# model=AutoModelForCausalLM.from_pretrained(
#     model_path, 
#     device_map='cpu',
#     trust_remote_code=True
# )
# count_parameters(model)

# # Deepseek-Coder-v2-Lite-Instruct_Fused_2B

sizes

2 = 2B

4 = 2.3B

8 = 3B

16 = 5B

In [ ]:
# Deepseek-Coder-V2-Lite-Instruct-Fused-2E-2B-preview-Unhealed
# Deepseek-Coder-V2-Lite-Instruct-Fused-4E-2_5B-preview-Unhealed
# Deepseek-Coder-V2-Lite-Instruct-Fused-8E-3B-preview-Unhealed
# Deepseek-Coder-V2-Lite-Instruct-Fused-16E-5B-preview-Unhealed

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] ="1"

In [ ]:
from datasets import load_dataset
from gptqmodel import GPTQModel, QuantizeConfig

model_id = "../../pruned_models/Deepseek-Coder-V2-Lite-Instruct-Fused-4E-2_5B-preview/"
quant_path = "pruned_models/Deepseek-Coder-V2-Lite-Instruct-Fused-4E-2_5B-preview-GPTQ"

calibration_dataset = load_dataset(
    "stas/openwebtext-10k",
    split="train"
  ).select(range(128))["text"]

calibration_dataset=[elt[:2048] for elt in calibration_dataset]

quant_config = QuantizeConfig(bits=4, group_size=128)

model = GPTQModel.load(
    model_id,
    quant_config,
    trust_remote_code=True,
)

# increase `batch_size` to match gpu/vram specs to speed up quantization


In [ ]:
model.quantize(calibration_dataset, batch_size=4)

model.save(quant_path)

# test post-quant inference
model = GPTQModel.load(quant_path)


In [ ]:
result = model.generate("Uncovering deep insights begins with")[0] # tokens
print(model.tokenizer.decode(result)) # string output